In [1]:
# Left rotate
def lrot(byte_string, offset):
    for i in range(offset):
        byte_string = byte_string[1:]+byte_string[0]
    return byte_string


# Transfer num to binary (string), e.g "00101011"
def b(num, bits=8):
    if -num > 2**(bits-1) or num > 2**(bits-1) - 1:
        return format(num, "b")[-bits:]

    maximum = 2 ** bits - 1
    if num < 0:
        n = format((-num - 1) ^ maximum, "b")
        if bits - len(n) > 0:
            return "1"*(bits - len(n)) + n
        else:
            return n
    else:
        n = format(num, "b")
        if bits - len(n) > 0:
            return "0"*(bits - len(n)) + n
        else:
            return n


# Transfer binary to num.
def to_num(input_bits):
    return int(input_bits, 2)


# Get a whole dict of words
def first_part_to_80_words(string):
    chunked_dict = {}
    s = ""
    for i in string:
        s += b(ord(i))  # String to ASCII.
    org_len = len(s)  # Original length of input string
    s += "1"  # Append '1'.
    amount = (512 * ((len(s) // 512) + 1) - (len(s) - 448)) % 512  # Append some zero's to make s congruent to 448 % 512.
    s += "0" * amount
    s += b(org_len, bits=64)  # Append Original length of input string in 64-bit.
    for x in range(len(s)//512):  # One chunk deal with up to 512 bit.
        chunked_dict[x] = {}
        for y in range(16):
            chunked_dict[x][y] = s[0:32]  # Store words (32-bit).
            s = s[32:]

    for i in chunked_dict.keys():  # Extend to 80 words.
        ini = 16
        for j in range(ini, 80):
            # d[i-3], d[i-8], d[i-14], d[i-16]
            xor_res = to_num(chunked_dict[i][j-3]) ^ to_num(chunked_dict[i][j-8])
            xor_res = xor_res ^ to_num(chunked_dict[i][j-14])
            xor_res = xor_res ^ to_num(chunked_dict[i][j-16])
            new_bits = b(xor_res, bits=32)
            new_bits = new_bits[1:] + new_bits[0]
            chunked_dict[i][j] = new_bits
    return chunked_dict


# Main loop
def hash_sha1(string):
    def function_1(x, y, z):
        f = b((x & y) | (~x & z), 32)
        k = "01011010100000100111100110011001"
        return f, k

    def function_2(x, y, z):
        f = b((x ^ y) ^ z, 32)
        k = "01101110110110011110101110100001"
        return f, k

    def function_3(x, y, z):
        f = b((x & y) | (x & z) | (y & z), 32)
        k = "10001111000110111011110011011100"
        return f, k

    def function_4(x, y, z):
        f = b((x ^ y) ^ z, 32)
        k = "11001010011000101100000111010110"
        return f, k

    print("Our input data is ","「",string,"」")
    h0 = "01100111010001010010001100000001"
    h1 = "11101111110011011010101110001001"
    h2 = "10011000101110101101110011111110"
    h3 = "00010000001100100101010001110110"
    h4 = "11000011110100101110000111110000"
    
    byte_dict = first_part_to_80_words(string)
    for key in byte_dict.keys():
        A = h0
        B = h1
        C = h2
        D = h3
        E = h4

        for i in range(80):
            if i < 20:
                F, K = function_1(to_num(B), to_num(C), to_num(D))
            elif i < 40:
                F, K = function_2(to_num(B), to_num(C), to_num(D))
            elif i < 60:
                F, K = function_3(to_num(B), to_num(C), to_num(D))
            elif i < 80:
                F, K = function_4(to_num(B), to_num(C), to_num(D))

            temp = to_num(lrot(A, 5)) + to_num(F) + to_num(E) + to_num(K) + to_num(byte_dict[key][i])
            temp = b(temp, 32)

            E = D
            D = C
            C = lrot(B, 30)
            B = A
            A = temp

        h0 = b((to_num(h0) + to_num(A)), 32)
        h1 = b((to_num(h1) + to_num(B)), 32)
        h2 = b((to_num(h2) + to_num(C)), 32)
        h3 = b((to_num(h3) + to_num(D)), 32)
        h4 = b((to_num(h4) + to_num(E)), 32)

    s0 = hex(int(h0, 2))[2:]
    s1 = hex(int(h1, 2))[2:]
    s2 = hex(int(h2, 2))[2:]
    s3 = hex(int(h3, 2))[2:]
    s4 = hex(int(h4, 2))[2:]
    print(end="\n")
    print("After hash function", end="\n")
    print("Digest/Hashes number is ", end="\n")
    print("-->",s0 + s1 + s2 + s3 + s4)
    #return s0 + s1 + s2 + s3 + s4


In [2]:
hash_sha1("TEST")

Our input data is  「 TEST 」

After hash function
Digest/Hashes number is 
--> 984816fd329622876e14907634264e6f332e9fb3


In [3]:
hash_sha1("Hash")

Our input data is  「 Hash 」

After hash function
Digest/Hashes number is 
--> 873507a022b58de26a88deae87268cbd8d6af5b1


In [4]:
hash_sha1("professor rusell is a good teacher")

Our input data is  「 professor rusell is a good teacher 」

After hash function
Digest/Hashes number is 
--> 6d5ba4336157f9f9f5dd5e9af5598827ac147700
